In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from osgeo import gdal
import numpy as np
import tensorflow as tf

In [38]:
side = 64
file_names = [
    "data/2017-01-01_MOD09GA_background_removal_zero_inputated_image_with_cf_50perc_grid_size10-0000017664-0000000000.tif",
]

In [136]:
def read_tiffs():
    
    for f in file_names:
        data = gdal.Open(f)
        rows = data.RasterXSize
        cols = data.RasterYSize

        rows -= rows % side
        cols -= cols % side

        for xoff in range(0, rows, side):
            for yoff in range(0, rows, side):
                bands = []
                for b in range(data.RasterCount):
                    band = data.GetRasterBand(b+1).ReadAsArray(
                        xoff,
                        yoff,
                        win_xsize=side,
                        win_ysize=side
                    )
                    bands.append(band)
                    
                if all(band is not None for band in bands):
                    yield np.stack(bands, axis=-1)

In [137]:
p = (tf.data.Dataset
    .from_generator(read_tiffs, tf.int16, tf.TensorShape((side, side, 7)))
    .apply(tf.contrib.data.shuffle_and_repeat(1000))
    .batch(32))

In [138]:
x = p.make_one_shot_iterator().get_next()

In [139]:
print("opening session...")
with tf.Session() as sess:
    print("initializing global variables...")
    sess.run(tf.global_variables_initializer())
    print("running pipeline...")
    y = sess.run(x)

opening session...
initializing global variables...
running pipeline...


In [141]:
y.shape

(32, 64, 64, 7)

In [ ]:
y.shape

In [ ]:
img = tf.placeholder( tf.int16, [64 * 10, 64 * 13, 7])

In [ ]:
x = np.random.random((64 * 3, 64 * 13, 7))
x = np.split(x, 3)
x = [np.split(y, 13, axis=1) for y in x] 

In [ ]:
x[0][0].shape

In [ ]:
side = 64

def _reader(fname):
    data = gdal.Open(fname, gdal.GA_ReadOnly)
    xs = []
    for i in range(dataset.RasterCount):
        x = data.GetRasterBand(i+1).ReadAsArray()
        rows, cols = x.shape
        rows -= rows % side
        cols -= cols % side
        xs.append(x[:rows, :cols])
        
    return np.stack(xs, axis=2)

# Coercing for TF
reader = lambda fname: tuple(tf.py_func(_reader, [fname], [tf.int16]))

# def splitter(img, nrows, ncols):
#     chunks = []
#     print(img, nrows, ncols)
    
#     for row in tf.split(img, nrows, axis=0):
#         for sq in tf.split(row, ncols, axis=1):
#             sq.set_shape((64, 64, 7))
#             chunks.append(sq)
            
#     return tf.data.Dataset.from_tensor_slices(chunks)
    
    
def _split(big):
    
    rows, cols, _ = big.size
    row_chunks = rows // side
    col_chunks = cols // side
    
    chunks = []
    for row in np.split(big, row_chunks):
        chunks.extend(np.split(row, col_chunks, axis=1))
        
    return np.stack(chunks, axis = 4)

split = lambda big: tuple(tf.py_func(_split, [big], [tf.int16]))

read_and_split = lambda fname: tuple(tf.py_func(_read_and_split, [fname], tf.int16))

p = (tf.data.Dataset
    .from_tensor_slices(file_names)
    .map(reader)
    .apply(tf.contrib.data.shuffle_and_repeat(1000))
    .prefetch(tf.contrib.data.AUTOTUNE)
)

In [ ]:
x = p.make_one_shot_iterator().get_next()
x

In [ ]:
print("opening session...")
with tf.Session() as sess:
    print("initializing global variables...")
    sess.run(tf.global_variables_initializer())
    print("running pipeline...")
    y = sess.run(x)

In [ ]:
y.shape

In [ ]:
x.shape